In [1]:
from dataset.build_dataset import build_dataset
import mindspore.dataset as ds
import os
import numpy as np
from mindspore.mindrecord import FileWriter
import json

[WARNING] ME(25711:140251066578752,MainProcess):2021-09-26-20:35:00.317.18 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(25711:140251066578752,MainProcess):2021-09-26-20:35:00.393.12 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
FEATURES_FILE = "./data/train_data.npy"
features = np.load(FEATURES_FILE)
list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)

In [3]:
len(list_dict[0]['word_ids'])

180

In [4]:
list_dict[0]['word_ids']

[0,
 598,
 2661,
 222,
 5,
 9880,
 2708,
 2346,
 2082,
 11,
 504,
 4432,
 11,
 226,
 2126,
 10067,
 1470,
 116,
 2,
 2,
 29474,
 28108,
 6,
 5,
 334,
 34,
 10,
 4019,
 2048,
 4,
 497,
 1517,
 5,
 4326,
 6919,
 18,
 1637,
 31346,
 16,
 10,
 9030,
 9577,
 9,
 5,
 9880,
 2708,
 4,
 29261,
 11,
 760,
 9,
 5,
 4326,
 6919,
 8,
 2114,
 24,
 6,
 16,
 10,
 7621,
 9577,
 9,
 4845,
 19,
 3701,
 62,
 33161,
 19,
 5,
 7875,
 22,
 39043,
 1459,
 1614,
 1464,
 13292,
 4977,
 845,
 4130,
 7,
 5,
 4326,
 6919,
 16,
 5,
 26429,
 2426,
 9,
 5,
 25095,
 6924,
 4,
 29261,
 639,
 5,
 32394,
 2426,
 16,
 5,
 7461,
 26187,
 6,
 10,
 19035,
 317,
 9,
 9621,
 8,
 12456,
 4,
 85,
 16,
 10,
 24633,
 9,
 5,
 11491,
 26187,
 23,
 226,
 2126,
 10067,
 6,
 1470,
 147,
 5,
 9880,
 2708,
 2851,
 13735,
 352,
 1382,
 7,
 6130,
 6552,
 625,
 3398,
 208,
 22895,
 853,
 1827,
 11,
 504,
 4432,
 4,
 497,
 5,
 253,
 9,
 5,
 1049,
 1305,
 36,
 463,
 11,
 10,
 2228,
 516,
 14,
 15230,
 149,
 155,
 19638,
 8,
 5,
 2610,
 25336

In [6]:
from typing import Union, List


def _get_bucket_length(x, bts):
    x_len = len(x)
    for index in range(1, len(bts)):
        if bts[index - 1] < x_len <= bts[index]:
            return bts[index]
    return bts[0]


class Pad:
    """
    Pads the input data samples to the largest length.

    Args:
        max_length (int): The required length of the data. If the input data less than
            the max_length, it indicates we pad it to the max_length with the pad_val.
        pad_val (Union[float, int]): The padding value, default 0.
        buckets (List[int], Optional): Padding row to the length of buckets, default None.
        pad_right (bool): The position of the PAD. If True, it indicates we
        pad to the right side, while False indicates we pad to the left side, default True.

     """

    def __init__(self, max_length: int = 0, pad_val: Union[float, int] = 0, buckets: List[int] = None,
                 pad_right: bool = True):
        self.pad_val = pad_val
        self.max_length = max_length
        self.buckets = buckets
        self.pad_right = pad_right

    def __call__(self, data: List[int]) -> List[int]:
        """
        Args:
            data (List[int]): The input data.

        Returns:
            List[int]: The input data which has been pad to max_length.

        Examples:
            .. code-block:: python
            >>> pad = Pad(max_length=10)
            >>> input_data = [1, 2, 3, 4, 5, 6]
            >>> result = Pad(input_data)
            >>> print(result)
            (1,2,3,4,5,6,0,0,0,0)
        """
        if self.buckets is not None:
            self.max_length = _get_bucket_length(data, self.buckets)
        if self.pad_right:
            data = data + (self.max_length - len(data)) * [self.pad_val]
        else:
            data = (self.max_length - len(data)) * [self.pad_val] + data
        return data

    @staticmethod
    def padding(data: List[int], max_length: int, pad_val: Union[float, int] = 0, pad_right: bool = True) -> List[int]:
        """
        Args:
            data (List[int]): The input data.
            max_length (int): The required length of the data. If the input data less than
                the max_length, it indicates we pad it to the max_length with the pad_val.
            pad_val (Union[float, int]): The padding value, default 0.
            pad_right (bool): The position of the PAD. If True, it indicates we
            pad to the right side, while False indicates we pad to the left side, default True.

        Returns:
            List[int]: the input data which has been pad to max_length.
        """
        if pad_right:
            data = data + (max_length - len(data)) * [pad_val]
        else:
            data = (max_length - len(data)) * [pad_val] + data
        return data

In [7]:
SQUAD_MINDRECORD_FILE = "./data/train_features.mindrecord"
pad_1= Pad(max_length=512,pad_val=1)
pad_0= Pad(max_length=512,pad_val=0)
pad_n1=Pad(max_length=128,pad_val=-1)
pad_entity = lambda a,i : a[0:i] if len(a) > i else np.append(a,[-1] * (i-len(a)))
for slist in list_dict:
    slist["entity_attention_mask"] = pad_128(slist["entity_attention_mask"])
    slist["entity_ids"] = pad_128(slist["entity_attention_mask"])
    slist["entity_segment_ids"] = pad_128(slist["entity_segment_ids"])
    
    slist["word_ids"] = pad_1(slist["word_ids"])
    slist["word_segment_ids"] = pad_0(slist["word_segment_ids"])
    slist["word_attention_mask"] = pad_0(slist["word_attention_mask"])
    slist["entity_position_ids"] = np.array(slist["entity_position_ids"]).flatten()
    slist["entity_position_ids"] = pad_entity(slist["entity_position_ids"])


if os.path.exists(SQUAD_MINDRECORD_FILE):
    os.remove(SQUAD_MINDRECORD_FILE)
    os.remove(SQUAD_MINDRECORD_FILE + ".db")

writer = FileWriter(file_name=SQUAD_MINDRECORD_FILE, shard_num=1)

data_schema = {
    "unique_id": {"type": "int32", "shape": [-1]},
    "word_ids": {"type": "int32", "shape": [-1]},
    "word_segment_ids": {"type": "int32", "shape": [-1]},
    "word_attention_mask": {"type": "int32", "shape": [-1]},
    "entity_ids": {"type": "int32", "shape": [-1]},
    "entity_position_ids": {"type": "int32", "shape": [-1]},
    "entity_segment_ids": {"type": "int32", "shape": [-1]},
    "entity_attention_mask": {"type": "int32", "shape": [-1]},
    "start_positions": {"type": "int32", "shape": [-1]},
    "end_positions": {"type": "int32", "shape": [-1]}
}
writer.add_schema(data_schema, "it is a preprocessed squad dataset")

data = []
i = 0
for item in list_dict:
    i += 1
    sample = {
        "unique_id": np.array(item["unique_id"], dtype=np.int32),
        "word_ids": np.array(item["word_ids"], dtype=np.int32),
        "word_segment_ids": np.array(item["word_segment_ids"], dtype=np.int32),
        "word_attention_mask": np.array(item["word_attention_mask"], dtype=np.int32),
        "entity_ids": np.array(item["entity_ids"], dtype=np.int32),
        "entity_position_ids": np.array(item["entity_position_ids"], dtype=np.int32),
        "entity_segment_ids": np.array(item["entity_segment_ids"], dtype=np.int32),
        "entity_attention_mask": np.array(item["entity_attention_mask"], dtype=np.int32),
        "start_positions": np.array(item["start_positions"], dtype=np.int32),
        "end_positions": np.array(item["end_positions"], dtype=np.int32),
    }

    data.append(sample)
    #print(sample)
    if i % 10 == 0:
        writer.write_raw_data(data)
        data = []
print(data[0])
if data:
    writer.write_raw_data(data)

writer.commit()

ValueError: operands could not be broadcast together with shapes (90,) (38,) 

In [ ]:
output_features = np.array(list_dict)
#SQUAD_MINDRECORD_FILE = "./data/dev_features.mindrecord"
data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
count = 0
for item in data_set.create_dict_iterator():
    #print(item)
    count += 1
print("Got {} samples".format(count))

In [ ]:
data_set = data_set.batch(1)
data_sample = next(data_set.create_dict_iterator())
data_sample

## Train

In [ ]:
from readingcomprehension.models.luke import LukeForReadingComprehension, LukeEntityAwareAttentionModel, LukeSquadCell
import mindspore.common.dtype as mstype
from model.bert_model import BertConfig
from mindspore import context
from model.luke import LukeModel, EntityAwareEncoder
from mindspore import Tensor, context
from mindspore import dtype as mstype
import mindspore.ops as ops
import mindspore.nn as nn
from model.bert_model import BertOutput
from mindspore.common.initializer import TruncatedNormal
from mindspore.ops import composite as C
import mindspore
from mindspore.ops import operations as P
from mindspore.train.model import Model
from tqdm import tqdm
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
import collections

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")
context.set_context(enable_graph_kernel=True)

In [ ]:

def do_train(dataset=None, network=None, load_checkpoint_path="", save_checkpoint_path="", epoch_num=1):
    """ do train """
    if load_checkpoint_path == "":
        raise ValueError("Pretrain model missed, finetune task must load pretrain model!")
    steps_per_epoch = dataset.get_dataset_size()
    # optimizer
    if optimizer_cfg.optimizer == 'AdamWeightDecay':
        lr_schedule = BertLearningRate(learning_rate=15e-6,
                                       end_learning_rate=optimizer_cfg.AdamWeightDecay.end_learning_rate,
                                       warmup_steps=int(steps_per_epoch * epoch_num * 0.1),
                                       decay_steps=steps_per_epoch * epoch_num,
                                       power=optimizer_cfg.AdamWeightDecay.power)
        params = network.trainable_params()
        decay_params = list(filter(optimizer_cfg.AdamWeightDecay.decay_filter, params))
        other_params = list(filter(lambda x: not optimizer_cfg.AdamWeightDecay.decay_filter(x), params))
        group_params = [{'params': decay_params, 'weight_decay': optimizer_cfg.AdamWeightDecay.weight_decay},
                        {'params': other_params, 'weight_decay': 0.0}]

        optimizer = AdamWeightDecay(group_params, lr_schedule, eps=optimizer_cfg.AdamWeightDecay.eps)
    elif optimizer_cfg.optimizer == 'Lamb':
        lr_schedule = BertLearningRate(learning_rate=optimizer_cfg.Lamb.learning_rate,
                                       end_learning_rate=optimizer_cfg.Lamb.end_learning_rate,
                                       warmup_steps=int(steps_per_epoch * epoch_num * 0.1),
                                       decay_steps=steps_per_epoch * epoch_num,
                                       power=optimizer_cfg.Lamb.power)
        optimizer = Lamb(network.trainable_params(), learning_rate=lr_schedule)
    elif optimizer_cfg.optimizer == 'Momentum':
        optimizer = Momentum(network.trainable_params(), learning_rate=optimizer_cfg.Momentum.learning_rate,
                             momentum=optimizer_cfg.Momentum.momentum)
    else:
        raise Exception("Optimizer not supported. support: [AdamWeightDecay, Lamb, Momentum]")

    # load checkpoint into network
    ckpt_config = CheckpointConfig(save_checkpoint_steps=steps_per_epoch, keep_checkpoint_max=1)
    ckpoint_cb = ModelCheckpoint(prefix="squad",
                                 directory=None if save_checkpoint_path == "" else save_checkpoint_path,
                                 config=ckpt_config)
    param_dict = load_checkpoint(load_checkpoint_path)
    load_param_into_net(network, param_dict)

    update_cell = DynamicLossScaleUpdateCell(loss_scale_value=2 ** 32, scale_factor=2, scale_window=1000)
    netwithgrads = BertSquadCell(network, optimizer=optimizer, scale_update_cell=update_cell)
    model = Model(netwithgrads)
    callbacks = [TimeMonitor(dataset.get_dataset_size()), LossCallBack(dataset.get_dataset_size()), ckpoint_cb]
    model.train(epoch_num, dataset, callbacks=callbacks)

In [ ]:
from mindspore.nn.learning_rate_schedule import LearningRateSchedule, PolynomialDecayLR, WarmUpLR
class BertLearningRate(LearningRateSchedule):
    """
    Warmup-decay learning rate for Bert network.
    """
    def __init__(self, learning_rate, end_learning_rate, warmup_steps, decay_steps, power):
        super(BertLearningRate, self).__init__()
        self.warmup_flag = False
        if warmup_steps > 0:
            self.warmup_flag = True
            self.warmup_lr = WarmUpLR(learning_rate, warmup_steps)
        self.decay_lr = PolynomialDecayLR(learning_rate, end_learning_rate, decay_steps, power)
        self.warmup_steps = Tensor(np.array([warmup_steps]).astype(np.float32))

        self.greater = P.Greater()
        self.one = Tensor(np.array([1.0]).astype(np.float32))
        self.cast = P.Cast()

    def construct(self, global_step):
        decay_lr = self.decay_lr(global_step)
        if self.warmup_flag:
            is_warmup = self.cast(self.greater(self.warmup_steps, global_step), mstype.float32)
            warmup_lr = self.warmup_lr(global_step)
            lr = (self.one - is_warmup) * decay_lr + is_warmup * warmup_lr
        else:
            lr = decay_lr
        return lr

In [ ]:
from mindspore.train.callback import Callback
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
import collections
import time
def get_ms_timestamp():
    t = time.time()
    return int(round(t * 1000))

class LossCallBack(Callback):
    def __init__(self, per_print_times=1,rank_ids=0):
        super(LossCallBack,self).__init__()
        if not isinstance(per_print_times, int) or per_print_times < 0:
            raise ValueError("print_step must be int and >=0.")
        self._per_print_times = per_print_times
        self.rank_id = rank_ids
        self.time_stamp_first = get_ms_timestamp()
        
    def step_end(self, run_context):
        """Monitor the loss in training."""
        global time_stamp_first
        time_stamp_current = get_ms_timestamp()
        cb_params = run_context.original_args()
        print("time: {}, epoch: {}, step: {}, outputs are {}".format(time_stamp_current - time_stamp_first,
                                                                     cb_params.cur_epoch_num,
                                                                     cb_params.cur_step_num,
                                                                     str(cb_params.net_outputs)))
        with open("./loss_{}.log".format(self.rank_id), "a+") as f:
            f.write("time: {}, epoch: {}, step: {}, loss: {}".format(
                time_stamp_current - time_stamp_first,
                cb_params.cur_epoch_num,
                cb_params.cur_step_num,
                str(cb_params.net_outputs.asnumpy())))
            f.write('\n')

In [ ]:
epoch=1

luke_config = BertConfig()
LUKEModel = LukeForReadingComprehension(luke_config)
param_dict = load_checkpoint('./luke-large-qa.ckpt')
load_param_into_net(LUKEModel,param_dict)

#lr_schedule = BertLearningRate()
params = LUKEModel.trainable_params()
optimizer = mindspore.nn.AdamWeightDecay(params,learning_rate=15e-6, beta1=0.9,beta2=0.98,eps=1e-06)

# lr_schedule
warmup_steps=877
num_train_steps=14629

netwithgrads = LukeSquadCell(LUKEModel,optimizer=optimizer)
model =Model(netwithgrads)

loss_monitor = LossCallBack()
model.train(epoch,data_set,callbacks=[loss_monitor],dataset_sink_mode=False)


